In [ ]:
#setup
!pip install geopandas rasterio earthengine-api folium

In [4]:
### bring in data ###
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
from datetime import datetime
import folium
from folium import plugins

# authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='colonia-detection')

# load colonias
url = "https://raw.githubusercontent.com/wvg1/colonia-detection/main/data/raw/COLONIAS_COMMUNITIES_-2949956472111660887.geojson"
colonias_gdf = gpd.read_file(url)

print(f"Total colonias: {len(colonias_gdf)}")
print(f"States: {colonias_gdf['STUSAB'].unique()}")

# filter for Texas
texas_colonias = colonias_gdf[colonias_gdf['STUSAB'] == 'TX']
print(f"Colonias in Texas: {len(texas_colonias)}")

# for Hidalgo specifically, use bounding box overlap
hidalgo_bbox = [-98.583, 26.033, -97.850, 26.767]
hidalgo_colonias = texas_colonias[
    (texas_colonias.geometry.x >= hidalgo_bbox[0]) &
    (texas_colonias.geometry.x <= hidalgo_bbox[2]) &
    (texas_colonias.geometry.y >= hidalgo_bbox[1]) &
    (texas_colonias.geometry.y <= hidalgo_bbox[3])
]
print(f"Colonias in Hidalgo bounding box: {len(hidalgo_colonias)}")

# query Sentinel-2 (same as before)
hidalgo_bbox = [-98.583, 26.033, -97.850, 26.767]
geometry = ee.Geometry.Rectangle([hidalgo_bbox[0], hidalgo_bbox[1], hidalgo_bbox[2], hidalgo_bbox[3]])

sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(geometry) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .sort('CLOUD_COVERAGE_ASSESSMENT') \
    .first()

image_info = sentinel2.getInfo()
props = image_info['properties']

date_ms = props['system:time_start']
date = datetime.fromtimestamp(date_ms / 1000).strftime('%Y-%m-%d')

print(f"\nSentinel-2 Image: {date} | Cloud cover: {props['CLOUDY_PIXEL_PERCENTAGE']}%")

Total colonias: 2008
States: ['AZ' 'TX' 'NM' 'CA']
Colonias in Texas: 1808
Colonias in Hidalgo bounding box: 932

Sentinel-2 Image: 2024-09-30 | Cloud cover: 0%


In [5]:
# create an interactive map centered on Hidalgo
center_lat = (hidalgo_bbox[1] + hidalgo_bbox[3]) / 2
center_lon = (hidalgo_bbox[0] + hidalgo_bbox[2]) / 2

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='OpenStreetMap'
)

# add colonias as points
for idx, row in hidalgo_colonias.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        popup=row['NAME'],
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.7
    ).add_to(m)

# add bounding box as rectangle
folium.Rectangle(
    bounds=[[hidalgo_bbox[1], hidalgo_bbox[0]], [hidalgo_bbox[3], hidalgo_bbox[2]]],
    color='blue',
    fill=False,
    weight=2,
    popup='Study Area'
).add_to(m)

m.save('hidalgo_colonias_map.html')
print("Map saved as hidalgo_colonias_map.html")
m

Map saved as hidalgo_colonias_map.html


In [6]:
# get the Sentinel-2 image URL for visualization
image = sentinel2.select(['B4', 'B3', 'B2'])  # RGB bands

# create a visualization
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# get the map tile URL
map_id = ee.Image(image).getMapId(vis_params)
tile_layer = map_id['tile_fetcher'].url_format

# create map
center_lat = (hidalgo_bbox[1] + hidalgo_bbox[3]) / 2
center_lon = (hidalgo_bbox[0] + hidalgo_bbox[2]) / 2

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles='OpenStreetMap'
)

# add Sentinel-2 imagery layer
folium.TileLayer(
    tiles=tile_layer,
    attr='Sentinel-2',
    name='Sentinel-2 RGB',
    overlay=False,
    control=True
).add_to(m)

# add colonias on top
for idx, row in hidalgo_colonias.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        popup=row['NAME'],
        color='yellow',
        fill=True,
        fillColor='yellow',
        fillOpacity=0.8,
        weight=2
    ).add_to(m)

folium.LayerControl().add_to(m)
m.save('hidalgo_sentinel2_colonias.html')
print("Satellite map with colonias saved")
m

Satellite map with colonias saved!
